In [45]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import warnings
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings('ignore')

def get_file_names(folder_path):
    files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    return files

missed_data_path = 'data/missed'
missed_data_points = get_file_names(missed_data_path)

passive_data_path = 'data/passive'
passive_data_points = get_file_names(passive_data_path)

In [46]:
most_prob = np.load('most_prob_region.npy')

In [47]:
region = ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"]

In [48]:
aca = np.isin(most_prob, region)
aca.sum()

383

In [49]:
missed_data = np.load(f'{missed_data_path}/{missed_data_points[1]}')

In [50]:
missed_data[:,aca]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.]])

In [51]:
class EEGFilesDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = []
        self.labels = []

        # Read files from 'missed' directory
        missed_dir = os.path.join(data_dir, 'missed')
        for file in os.listdir(missed_dir):
            if file.endswith('.npy'):
                self.files.append(os.path.join(missed_dir, file))
                self.labels.append(0)  # Label for 'missed'

        # Read files from 'passive' directory
        passive_dir = os.path.join(data_dir, 'passive')
        for file in os.listdir(passive_dir):
            if file.endswith('.npy'):  # Change file extension based on your data format
                self.files.append(os.path.join(passive_dir, file))
                self.labels.append(1)  # Label for 'passive'

        # Initialize a scaler for normalization
        self.scaler = StandardScaler()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # Load data from file
        data = np.load(self.files[idx])[:,aca]

        # Flatten the data if it's 2D (you might need to adjust this based on your data shape)
        if data.ndim > 1:
            data = data.flatten()

        # Normalize the data
        data = self.scaler.fit_transform(data.reshape(-1, 1)).flatten()

        # Convert data and label to torch tensors
        return torch.tensor(data, dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Create the dataset
dataset = EEGFilesDataset('data')

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [52]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.act = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act(self.layer1(x))
        x = self.sigmoid(self.layer2(x))  # This layer maps to the number of categories
        return x

# Adjust these parameters according to your needs
input_size = 19150  # Change this based on your actual input size
hidden_size = 100   # You can adjust the size of the hidden layer
output_size = 1     # Assuming you have two categories

net = SimpleNN(input_size, hidden_size, output_size)


In [53]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = net(inputs).squeeze()
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


Epoch 1/10, Loss: 0.16532109677791595
Epoch 2/10, Loss: 0.006952621508389711
Epoch 3/10, Loss: 0.0036087092012166977
Epoch 4/10, Loss: 0.00021614525758195668
Epoch 5/10, Loss: 0.00036162909236736596
Epoch 6/10, Loss: 2.9559860195149668e-05
Epoch 7/10, Loss: 0.00015791492478456348
Epoch 8/10, Loss: 0.00017291745461989194
Epoch 9/10, Loss: 0.0005982833681628108
Epoch 10/10, Loss: 6.555828440468758e-05
